# Azure AI Search az NVIDIA NIM és LlamaIndex integrációjával

Ebben a notebookban bemutatjuk, hogyan lehet az NVIDIA AI modelljeit és a LlamaIndexet felhasználni egy hatékony Retrieval-Augmented Generation (RAG) folyamat létrehozásához. Az NVIDIA LLM-jeit és embeddingjeit fogjuk használni, integrálva azokat az Azure AI Search szolgáltatással, mint vektortárolóval, és RAG-ot végrehajtva a keresési minőség és hatékonyság javítása érdekében.

## Előnyök
- **Skálázhatóság**: Használja ki az NVIDIA nagy nyelvi modelljeit és az Azure AI Search szolgáltatást a skálázható és hatékony lekérdezés érdekében.
- **Költséghatékonyság**: Optimalizálja a keresést és a lekérdezést hatékony vektortárolással és hibrid keresési technikákkal.
- **Magas teljesítmény**: Kombinálja az erőteljes LLM-eket a vektorizált kereséssel a gyorsabb és pontosabb válaszok érdekében.
- **Minőség**: Tartsa fenn a magas keresési minőséget az LLM válaszainak releváns dokumentumokkal való alátámasztásával.

## Előfeltételek
- 🐍 Python 3.9 vagy újabb
- 🔗 [Azure AI Search szolgáltatás](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API kulcs az NVIDIA LLM-ekhez és embeddingekhez való hozzáféréshez az NVIDIA NIM mikroszolgáltatásokon keresztül

## Lefedett funkciók
- ✅ NVIDIA LLM integráció (a [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) modellt fogjuk használni)
- ✅ NVIDIA embeddingek (a [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) modellt fogjuk használni)
- ✅ Azure AI Search fejlett lekérdezési módok
- ✅ Dokumentumindexelés a LlamaIndex segítségével
- ✅ RAG az Azure AI Search és a LlamaIndex használatával, NVIDIA LLM-ekkel

Kezdjünk neki!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Telepítés és követelmények
Hozzon létre egy Python-környezetet Python 3.10 vagy újabb verzióval.

## Első lépések!


Ahhoz, hogy elkezdhesd, szükséged lesz egy `NVIDIA_API_KEY` kulcsra az NVIDIA AI Foundation modellek használatához:
1) Hozz létre egy ingyenes fiókot az [NVIDIA](https://build.nvidia.com/explore/discover) oldalán.
2) Kattints a választott modellre.
3) Az Input alatt válaszd ki a Python fület, majd kattints a **Get API Key** gombra, és utána a **Generate Key** gombra.
4) Másold ki és mentsd el a generált kulcsot NVIDIA_API_KEY néven. Ettől kezdve hozzáférsz az endpointokhoz.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG példa LLM és beágyazás használatával
### 1) Az LLM inicializálása
A `llama-index-llms-nvidia`, más néven az NVIDIA LLM csatlakozója, lehetővé teszi, hogy csatlakozz kompatibilis modellekhez és generálj az NVIDIA API katalógusában elérhető modellekből. Itt található a chat completion modellek listája: https://build.nvidia.com/search?term=Text-to-Text

Ebben a példában a **mixtral-8x7b-instruct-v0.1** modellt fogjuk használni.


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Az embedding inicializálása
A `llama-index-embeddings-nvidia`, más néven NVIDIA Embeddings csatlakozó, lehetővé teszi, hogy csatlakozz kompatibilis modellekhez, és generálj az NVIDIA API katalógusban elérhető modellekből. Az `nvidia/nv-embedqa-e5-v5` modellt választottuk embedding modellként. Itt található egy lista a szöveg embedding modellekről: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Hozzon létre egy Azure AI Search Vector Store-t


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Hozz létre egy lekérdezési motort, hogy kérdéseket tehess fel az adataiddal kapcsolatban

Itt van egy lekérdezés, amely tisztán vektorkeresést használ az Azure AI Search-ben, és a választ a LLM-ünkhöz (Phi-3.5-MOE) köti.


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Itt van egy lekérdezés hibrid keresés használatával az Azure AI Search-ben.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Vektorkeresés elemzése
Az LLM válasza pontosan megragadja az orosz tőzsdével kapcsolatos kulcsfontosságú gazdasági következményeket, amelyeket a forrásszöveg említ. Konkrétan megállapítja, hogy az orosz tőzsde jelentős visszaesést szenvedett el, értékének 40%-át elveszítette, és a kereskedést felfüggesztették a kialakult helyzet miatt. Ez a válasz jól összhangban van a forrásban megadott információkkal, jelezve, hogy az LLM helyesen azonosította és foglalta össze a tőzsdei hatásokkal kapcsolatos releváns részleteket, amelyek Oroszország lépései és a szankciók következményeként jelentkeztek.

#### Forráscsomópontok megjegyzései
A forráscsomópontok részletesen bemutatják azokat a gazdasági következményeket, amelyekkel Oroszország a nemzetközi szankciók miatt szembesült. A szöveg kiemeli, hogy az orosz tőzsde értékének 40%-át elveszítette, és a kereskedést felfüggesztették. Emellett más gazdasági hatásokat is említ, például a rubel leértékelődését és Oroszország gazdaságának szélesebb körű elszigetelődését. Az LLM válasza hatékonyan összefoglalta ezekből a csomópontokból a legfontosabb pontokat, különös tekintettel a tőzsdei hatásokra, ahogyan azt a kérdésben kérték.


Most nézzünk meg egy olyan lekérdezést, ahol a Hibrid Keresés nem ad megalapozott választ:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hibrid Keresés: LLM Válasz Elemzése
A Hibrid Keresés példájában az LLM válasza arra utal, hogy a megadott kontextus nem tartalmazza Oroszország Ukrajna elleni inváziójának pontos dátumát. Ez a válasz azt sugallja, hogy az LLM a forrásdokumentumokban elérhető információkra támaszkodik, miközben elismeri, hogy a szövegben nincsenek pontos részletek.

A válasz helyesen azonosítja, hogy a kontextus említi Oroszország agressziójával kapcsolatos eseményeket, de nem határozza meg az invázió konkrét dátumát. Ez jól mutatja az LLM képességét arra, hogy megértse a rendelkezésre álló információkat, miközben felismeri a tartalom hiányosságait. Az LLM hatékonyan ösztönzi a felhasználót arra, hogy külső forrásokat vagy történelmi feljegyzéseket keressen a pontos dátum érdekében, óvatosságot tanúsítva, amikor az információ hiányos.

### Forráscsomópontok Elemzése
A Hibrid Keresés példájában a forráscsomópontok egy beszéd részleteit tartalmazzák, amely az Egyesült Államok válaszát tárgyalja Oroszország Ukrajna elleni lépéseire. Ezek a csomópontok hangsúlyozzák a szélesebb geopolitikai hatásokat, valamint az Egyesült Államok és szövetségesei által az invázióra adott válaszlépéseket, de nem említik az invázió konkrét dátumát. Ez összhangban van az LLM válaszával, amely helyesen azonosítja, hogy a kontextusból hiányzik a pontos dátumra vonatkozó információ.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hibrid újrarangsorolással: LLM válaszelemzés
A Hibrid újrarangsorolás példájában az LLM válasza további kontextust nyújt azzal, hogy megjegyzi, az esemény hat nappal a beszéd elhangzása előtt történt. Ez azt jelzi, hogy az LLM képes következtetni az invázió dátumára a beszéd időzítése alapján, bár a pontosság érdekében továbbra is szüksége van a beszéd pontos dátumának ismeretére.

Ez a válasz javított képességet mutat arra, hogy kontextuális nyomokat használjon egy informatívabb válasz megadásához. Kiemeli az újrarangsorolás előnyét, amely lehetővé teszi az LLM számára, hogy hozzáférjen és előtérbe helyezze a relevánsabb információkat, így közelebb kerülhet a kívánt részlet (azaz az invázió dátuma) pontos meghatározásához.

### Forráscsomópontok elemzése
Ebben a példában a forráscsomópontok utalnak Oroszország inváziójának időzítésére, különösen megemlítve, hogy az hat nappal a beszéd előtt történt. Bár a pontos dátum továbbra sincs kifejezetten megadva, a csomópontok időbeli kontextust biztosítanak, amely lehetővé teszi az LLM számára, hogy árnyaltabb választ adjon. Ennek a részletnek a beépítése bemutatja, hogyan javíthatja az újrarangsorolás az LLM képességét arra, hogy a rendelkezésre álló kontextusból információt nyerjen ki és következtessen, ezáltal pontosabb és informatívabb válaszokat eredményezve.


**Megjegyzés:**
Ebben a jegyzetfüzetben az NVIDIA API Katalógusból származó NVIDIA NIM mikroszolgáltatásokat használtuk.  
A fent említett API-k, például `NVIDIA (llms)`, `NVIDIAEmbedding`, és [Azure AI Search Semantic Hybrid Retrieval (beépített újrarangsorolás)](https://learn.microsoft.com/azure/search/semantic-search-overview).  
Fontos megjegyezni, hogy a fent említett API-k önállóan üzemeltetett mikroszolgáltatásokat is támogatnak.  

**Példa:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")



---

**Felelősségkizárás**:  
Ez a dokumentum az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével készült. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt a professzionális, emberi fordítás igénybevétele. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
